In [57]:
import requests
import pandas as pd
import time

In [4]:
key = '72e4097c974d4ddebaa4a2a2674d70d3'
endpoint = "https://api.cognitive.microsoft.com/bing/v7.0/search"
headers = {"Ocp-Apim-Subscription-Key": key}
params = {"q": '', "textDecorations": False, "textFormat": "HTML", 'count': 3,'responseFilter':'Webpages',}

In [56]:
test_schools = pd.read_csv('scrape_school_list_v1_cleaned_with_correct_urls.csv', index_col = 0)
test_schools['zip']=test_schools['zip'].apply(lambda x: str(x).zfill(5))


In [33]:
#Currently created manually, might be better once we can run a lot of testing to just filter out domains that come up frequently
url_filters = ['facebook.com','elementaryschools.org','usnews.com','greatschools.org',
              'niche.com','schooldigger.com','wikipedia.org','publicschoolreview.com','areavibes.com','realtor.com']
database_sites = ['greatschools.org','niche.com','schooldigger.com','elementaryschools.org','publicschoolreview.com']

In [7]:
"DO NOT CHANGE, USED TO GET CORRECT URLS"
def name_city(row):
    search_term = row['cleaned']
    if row['city'] not in search_term:
        search_term += ' '+ row['city']
    return search_term

def name(row):
    search_term = row['cleaned']
    return search_term

In [22]:
def whitelisted(url_string):
    return any(website in url_string for website in url_filters)
def is_database(url_string):
    return any(website in url_string for website in databse_sites.values())

Might be improved by filtering out websites that are not correct and taking the first link not white listed.
We could also try looking at those white listed sites to get the link from them as they often have that information but we would need to be careful.

In [38]:
def get_url(row,searchTermFunc):
    params['q'] = searchTermFunc(row)
    response = requests.get(endpoint, headers=headers, params=params)
    search_results = response.json()
    urls = [search_results['webPages']['value'][i]['url'] for i in range(3)]
    #To prevent calling the API too many times per second
    time.sleep(.5)
    if not any(check in urls[0] for check in url_filters):
        return [urls[0],None]
    for url in urls:
        if any(check in url for check in database_sites):
            return [None,url]
    return [None,None]

In [39]:
#Be careful running this as it performs as many API calls as the test_schools_df and we have limited amounts of calls
results = test_schools_25.apply(get_url,args = [name_city],axis=1)
test_schools_25['url'],test_schools_25['alt_url'] = map(list, zip(*results)) 

In [10]:
"This was updated in the CSV and is no longer necesary"
# #Rename the colummn to go with the function uses
# test_schools_25 = test_schools_25.rename(columns = {'url': 'simple_url_name_city'})
# #Manually update the correct values 
# test_schools_25['correct_url'] = test_schools_25['simple_url_name_city']
# test_schools_25.at[7,'correct_url'] = 'https://meansacademy.org/'
# test_schools_25.at[9,'correct_url'] = 'https://www.guthrieps.net/vnews/display.v/SEC/GUES-Upper%20Elementary'
# test_schools_25.at[11,'correct_url'] = 'https://pierre.k12.sd.us/kennedy.html'
# test_schools_25.at[12,'correct_url'] = None
# test_schools_25.at[20,'correct_url'] = None
# test_schools_25.at[21,'correct_url'] = None
# test_schools_25.at[22,'correct_url'] = None

Notes on the simple name_url search and where correct urls came from
Maine Academy Of Natural Sciences should be https://meansacademy.org/, linked on site we found, not accesible via search
Guthrie Upper Elementary School went to the district home page, should go here: https://www.guthrieps.net/vnews/display.v/SEC/GUES-Upper%20Elementary, correct link was in top 3 results
Kennedy Elementary - 08 https://pierre.k12.sd.us/kennedy.html
Spring Grove Primary Center doesn't appear to have a site
Chewelah Open Doors Reengagement Program is not a normal school but it seemed to go to the right place
New Life Alternatives Christian Academy does not appear to have a site
Morrow Valley Christian Academy does not appear to have a site
Walnut Creek School does not appear to have a site



In [53]:
count = 0
test_schools_25.at[12,'correct_url'] = 'http://www.usd499.org/'

for i, row in test_schools_25.iterrows():
    url = row['correct_url']
    if url == 'None': url = None
    if url == row['correct_url']:
        count += 1
    else:
        print(i)


25


In [55]:
test_schools_25.loc[[25]]

,id,level,dist_id,district,school,state,county,city,address,zip,cleaned,correct_url,url,alt_url
random_order,,,,,,,,,,,,,,
25,Bb161222,Combined Elem/Sec,Nan,Nan,Evangel Christian Academy,LA,Caddo,Shreveport,5720 BUNCOMBE RD,71129,Evangel Christian Academy,https://www.evangelacademy.com/,https://www.evangelacademy.com/,None
